# Лабораторная 4 - предобработка текста

In [5]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
import pymorphy3
from tqdm import tqdm 
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
data = pd.read_csv('jumoresci_data.csv')
data.index = data.id_post
data = data.drop(columns = ['id_group', 'description', 'title', 'id_post', 'views', 'reposts'])
data = data.dropna()
data.head(3)

,data,text,likes
id_post,,,
480545,1713384533,ТЕЛЕГА: острополитическое\n \nhttps://t.me/myf...,117
480648,1713543900,- Павел\n- Павел\n- Доводилось ли вам бывать н...,63
480645,1713536700,У гея оптимиста сракотан всегда наполовину полон,236


для дальнейшей работы будет использовать лишь последние 3000 постов, чтобы изолировать влияние времени на данные

In [7]:
data = data.head(3000)

In [8]:
punctuations = list(punctuation)
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('russian')
morph = pymorphy3.MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vsevo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def preprocessing(text):
    tokens = word_tokenize(text)
    tokens_without_punct = [i for i in tokens if i not in punctuations]
    low_tokens = [i.lower() for i in tokens_without_punct]
    words_without_stop = [i for i in low_tokens if i not in stopwords]
    lemms = [morph.parse(i)[0].normal_form for i in words_without_stop]
    morphed_text = " ".join(lemms)
    return morphed_text

In [10]:
preprocessed_texts = []

for text in tqdm(data['text']):
    preprocessed_texts.append(preprocessing(text))

data['text_preprocessed'] = preprocessed_texts

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:13<00:00, 216.53it/s]


In [12]:
data['text_preprocessed'][:5]

id_post
480545    телега острополитический https //t.me/myfavori...
480648    павел павел доводиться бывать воля павел павел...
480645              гей оптимист сракотан наполовину полный
480642    изобрести новый ракета квантовый двигатель пос...
480641    владелец квартира спрашивать квартирантка поче...
Name: text_preprocessed, dtype: object

Специфичные для датасета стоп слова будут выявлены на этапе векторизации в зависимости от их частоты